In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import to_categorical

from scipy.signal import lfilter

from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

#--------------------------------------------------------------------------------------------------------


def iirfilter(x_train): # digital filter, to smooth the drift
    
    n = 1000  # the larger n is, the smoother the curve will be
    b = [1.0 / n] * n
    a = 1
    
    filtered_x_train = lfilter(b,a,x_train)
    
    return filtered_x_train

test = pd.read_csv('../input/liverpool-ion-switching/test.csv')       #length = 5000000
train = pd.read_csv('../input/liverpool-ion-switching/train.csv')     #length = 2000000

#shuffle the dataset
train = shuffle(train)
test = shuffle(test)

x_train = train.iloc[:,1] #signal values, column
y_train = train.iloc[:,2] #open_channel values, column

y_train = to_categorical(y_train)

x_train = iirfilter(x_train)

x_eval = x_train[:1000000]
y_eval = y_train[:1000000]

#convert to numpy array
x_train = np.array(x_train)
y_train = np.array(y_train)
x_eval = np.array(x_eval)
y_eval = np.array(y_eval)


#reshape necessary for use in MinMaxScaler
x_train = x_train.reshape(-1,1)
x_eval = x_eval.reshape(-1,1)
y_eval = y_eval.reshape(-1,1)


#data normalization
scaler = MinMaxScaler(feature_range = (0, 1))
x_train = scaler.fit_transform(x_train)
x_eval = scaler.fit_transform(x_eval)


#from 5000000 / 20000 = 250, 20k batches of 250
n_samples = 20000 
n_timesteps = 250 
n_features = 1

#reshaping to use in input_shape argument in LSTM
x_train = x_train.reshape(n_samples,n_timesteps,1)
y_train = y_train.reshape(n_samples,n_timesteps,11)
x_eval = x_eval.reshape(4000,250,1)
y_eval = y_eval.reshape(4000,250,11)



# LSTM architecture settings and run
def model_init():

    model = Sequential()

    model.add(LSTM(units=32, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
    model.add(Dense(units = 11, activation='softmax'))

    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

    model.fit(x = x_train, y = y_train, validation_data=(x_eval, y_eval), epochs = 3, batch_size = 250)
        
    print(model.summary())
    
    return model

    
trained_model = model_init()

Train on 20000 samples, validate on 4000 samples
Epoch 1/3
20000/20000 [==============================] - 24s 1ms/step - loss: 2.1680 - accuracy: 0.2290 - val_loss: 2.1090 - val_accuracy: 0.2466
Epoch 2/3
20000/20000 [==============================] - 24s 1ms/step - loss: 2.1077 - accuracy: 0.2467 - val_loss: 2.1076 - val_accuracy: 0.2467
Epoch 3/3
20000/20000 [==============================] - 23s 1ms/step - loss: 2.1068 - accuracy: 0.2468 - val_loss: 2.1068 - val_accuracy: 0.2467
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 250, 32)           4352      
_________________________________________________________________
dense_2 (Dense)              (None, 250, 11)           363       
Total params: 4,715
Trainable params: 4,715
Non-trainable params: 0
_________________________________________________________________
None
